<a href="https://colab.research.google.com/github/aditya04jain/draft_simulator/blob/main/project_closed_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [197]:
import random

class Team():
  def __init__(self,name,league):
      self.name = name
      self.league = league
      self.wins = 0
      self.losses = 0
      self.draws = 0
      self.win_factor = 0
      self.points = 0

  def set_win_factor(self,win_factor):
      self.win_factor = win_factor

  def set_record(self, wins, losses, draws, points):
      self.wins = wins
      self.losses = losses
      self.draws = draws
      self.points = points

  def update_points(self):
      self.points = self.wins * 3 + self.draws

class game():
    def __init__(self,team1,team2):
      self.team1 = team1
      self.team2 = team2

    def play_game(self):

      total_win_factor = self.team1.win_factor + self.team2.win_factor
      team1_win_probability = self.team1.win_factor / total_win_factor
      draw_margin = 0.03

      random_result = random.random()

      if abs(random_result - team1_win_probability) <= draw_margin:
          self.team1.draws += 1
          self.team2.draws += 1
          ## return self.team1.name  # Team1 wins
      elif random_result < team1_win_probability:
          self.team1.wins += 1
          self.team2.losses += 1
          ## return self.team2.name  # Team2 wins
      else:
          self.team2.wins += 1
          self.team1.losses += 1

In [198]:
team_list = [
    "Arsenal", "Villa", "Bournemouth", "Brentford",
    "Brighton", "Burnley", "Chelsea", "Crystal Palace",
    "Everton", "Fulham", "Liverpool", "Luton Town",
    "Manchester City", "Manchester United", "Newcastle",
    "Nottingham Forest", "Sheffield Utd", "Tottenham",
    "West Ham United", "Wolverhampton"
]


DIV1=[]
DIV2=[]
i = 0
for team_name in team_list:
  if (i < 10):
    team_name = Team(team_name,"DIV1")
    DIV1.append(team_name)
  else:
    team_name = Team(team_name, "DIV2")
    DIV2.append(team_name)
  i += 1

for team in DIV1:
  team.set_win_factor(0.40 + random.random() * 0.45)
  # print(team.win_factor)

for team in DIV2:
  team.set_win_factor(0.20 + random.random() * 0.3)
  # print(team.win_factor)

In [199]:
def play_season():

  for team2 in DIV1:
    for team1 in DIV1:
      if team2 != team1:
        new_game = game(team2,team1)
        new_game.play_game()

  for team2 in DIV2:
    for team1 in DIV2:
      if team2 != team1:
        new_game = game(team2,team1)
        new_game.play_game()

  for team in DIV1:
    team.update_points()

  for team in DIV2:
    team.update_points()

In [200]:
!pip install pandas
from google.colab import files
from IPython.display import HTML
import pandas as pd

In [201]:
all_seasons_rankings = pd.DataFrame()

def display_team_ranking(teams_div1, teams_div2, season, all_seasons_rankings):
    # Sorting the teams based on points, then wins, and then name as a tie-breaker for each division
    sorted_teams_div1 = sorted(teams_div1, key=lambda x: (x.points, x.wins, x.name), reverse=True)
    sorted_teams_div2 = sorted(teams_div2, key=lambda x: (x.points, x.wins, x.name), reverse=True)

    # Create DataFrames for each division
    data_div1 = {
        'Rank_DIV1': [rank for rank, _ in enumerate(sorted_teams_div1, start=1)],
        'Team_DIV1': [team.name for team in sorted_teams_div1],
        'Wins_DIV1': [team.wins for team in sorted_teams_div1],
        'Draws_DIV1': [team.draws for team in sorted_teams_div1],
        'Losses_DIV1': [team.losses for team in sorted_teams_div1],
        'Points_DIV1': [team.points for team in sorted_teams_div1],
        'W-F_DIV1': [round(team.win_factor, 2) for team in sorted_teams_div1],
    }

    data_div2 = {
        'Rank_DIV2': [rank for rank, _ in enumerate(sorted_teams_div2, start=1)],
        'Team_DIV2': [team.name for team in sorted_teams_div2],
        'Wins_DIV2': [team.wins for team in sorted_teams_div2],
        'Draws_DIV2': [team.draws for team in sorted_teams_div2],
        'Losses_DIV2': [team.losses for team in sorted_teams_div2],
        'Points_DIV2': [team.points for team in sorted_teams_div2],
        'W-F_DIV2': [round(team.win_factor, 2) for team in sorted_teams_div2],
    }

    ranking_df_div1 = pd.DataFrame(data_div1)
    ranking_df_div2 = pd.DataFrame(data_div2)

    # Create a single DataFrame by combining DIV1 and DIV2 rankings
    combined_ranking_df = pd.concat([ranking_df_div1, ranking_df_div2], axis=1)
    combined_ranking_df['Season'] = season  # Add a column for the season number

    # Append the season's ranking to the global DataFrame with 2 empty rows for separation
    if all_seasons_rankings.empty:
        all_seasons_rankings = combined_ranking_df
    else:
        # Append with two empty rows as separators
        empty_row = pd.DataFrame([[''] * combined_ranking_df.shape[1]], columns=combined_ranking_df.columns)
        all_seasons_rankings = pd.concat([all_seasons_rankings, empty_row, empty_row, combined_ranking_df], ignore_index=True)

    return all_seasons_rankings

In [202]:
def post_season():
    sorted_DIV1 = sorted(DIV1, key=lambda x: (x.points, x.wins, x.name))
    sorted_DIV2 = sorted(DIV2, key=lambda x: (x.points, x.wins, x.name))

    #bottom 3 teams in DIV1
    #top 3 teams in DIV2
    bottom_teams_DIV1 = sorted_DIV1[0:3]
    top_teams_DIV2 = sorted_DIV2[-3:]

    for team in sorted_DIV2:
       if team not in top_teams_DIV2:
        team.set_win_factor(team.win_factor+(-0.08+0.08*random.random()))

    for team in sorted_DIV1:
      if team not in bottom_teams_DIV1:
        position = sorted_DIV1.index(team) + 1
        factor = team.win_factor
        team.set_win_factor(team.win_factor+draft_bump(position))

    for team in bottom_teams_DIV1:
        DIV2.append(team)
        DIV1.remove(team)
        team.set_win_factor(team.win_factor-(0.20*random.random()))

    for team in top_teams_DIV2:
        position = sorted_DIV2.index(team) + 1
        DIV1.append(team)
        DIV2.remove(team)
        team.set_win_factor(team.win_factor+sharp_positive(position))

    # reset records for next season
    for team in DIV1 + DIV2:
        team.set_record(0, 0, 0, 0)

In [203]:
def sharp_positive(position):
  return 1 / (position + (10 + 7 * random.random()))

In [204]:
def draft_bump (position):
  return 1 / (position + (100 + 100*random.random()))

In [205]:
def controller(all_seasons_rankings):
    for season in range(1, 6):  # Simulate 5 seasons
        play_season()
        all_seasons_rankings = display_team_ranking(DIV1, DIV2, season, all_seasons_rankings)
        post_season()

    # Export the combined DataFrame to a CSV file
    filename = all_seasons_rankings.to_csv('All_Seasons_Combined_Rankings.csv', index=False)
    files.download('All_Seasons_Combined_Rankings.csv')

In [206]:
controller(all_seasons_rankings)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>